In [8]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")

!pip install -r requirements.txt  --upgrade --user

  Using cached kfp-1.8.9-py3-none-any.whl


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

from typing import NamedTuple
def tf_func(log_folder:str) -> NamedTuple('Outputs', [('logdir', str)]):
    
    print('tf_func:', log_folder)
    import numpy as np  
    import pandas as pd  
    from tensorflow.python.keras.utils.np_utils import to_categorical
    import os
    import seaborn as sns
    import cv2
    import tensorflow
    from tensorflow.keras.models import Sequential  
    from tensorflow.keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization
    import time
    import tensorflow.keras
    from sklearn.model_selection import train_test_split
    
    
    def data_img(dirdata1,dirdata2,dirdata3,dirdata4):
        data = [] 
        label = []
        height=120
        BLACK = [0,0,0]
    for i in os.listdir(dirdata1): 
        img = cv2.imread(dirdata1+i)
        x,y,z = img.shape
        if x<=height:
            constant = cv2.copyMakeBorder(img,0,height-x,0,0,cv2.BORDER_CONSTANT,value=BLACK)
        data.append(constant) 
        label.append(0) 
    for i in os.listdir(dirdata2):
        img = cv2.imread(dirdata2+i)
        x,y,z = img.shape
        if x<=height:
            constant = cv2.copyMakeBorder(img,0,height-x,0,0,cv2.BORDER_CONSTANT,value=BLACK)
        data.append(constant) 
        label.append(1) 
        
    for i in os.listdir(dirdata3):
        img = cv2.imread(dirdata3+i)
        x,y,z = img.shape
        if x<=height:
            constant = cv2.copyMakeBorder(img,0,height-x,0,0,cv2.BORDER_CONSTANT,value=BLACK)
        data.append(constant) 
        label.append(2)
        
    for i in os.listdir(dirdata4):
        img = cv2.imread(dirdata4+i)
        x,y,z = img.shape
        if x<=height:
            constant = cv2.copyMakeBorder(img,0,height-x,0,0,cv2.BORDER_CONSTANT,value=BLACK)
        data.append(constant) 
        label.append(3) 
        
    data = np.array(data) 
    label = np.array(label) 

    label_one_hot = np.eye(4)[label] 

    data_norm = data / 255 
    return data_norm,label_one_hot

    dirdata1='0_img/'
    dirdata2='1_img/'
    dirdata3='2_img/'
    dirdata4='3_img/'
    data, label=data_img(dirdata1,dirdata2,dirdata3,dirdata4)
    

    X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.5, random_state=42)
    from tensorflow.keras.applications import vgg16
    vgg16=vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(120,50,3))
    model = Sequential()
    
    for layer in vgg16.layers:
        model.add(layer)
        model.summary()
    for layer in model.layers:
        layer.trainable = False
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.25))
        model.add(Dense(4, activation='softmax'))
        model.summary()
    
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])  

    start_time = time.time()
    train_history = model.fit(X_train,Y_train,validation_split=0.2,epochs=100, batch_size=30, verbose=1)
    end_time = time.time()
    
    import datetime
    import os
    
    ### add tensorboard logout callback
    log_dir = os.path.join(log_folder, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    ######
    
    model.fit(x=x_train, 
              y=y_train, 
              epochs=5, 
              validation_data=(x_test, y_test), 
              callbacks=[tensorboard_callback])

    print('At least tensorboard callbacks are correct')
    print('logdir:', log_dir)
    return ([log_dir])

def tensorboard_func(log_dir:str) -> NamedTuple('Outputs', [('mlpipeline_ui_metadata', 'UI_metadata')]):
    
    print('tensorboard_func:', log_dir)

    metadata = {
      'outputs' : [{
        'type': 'tensorboard',
        'source': 'gs://footprintai-kubeflow-workshop/mnist/epoch5',
      }]
    }
    import json
    return ([json.dumps(metadata)])

In [10]:
import kfp.dsl as dsl
import kfp.components as components

@dsl.pipeline(
   name='vgg16 pipeline',
   description='A pipeline to train a model on vgg16 dataset and start a tensorboard.'
)
def vgg16_pipeline():

    log_folder = '/persist-log'
    pvc_name = "mypvc"
    
    vop = dsl.VolumeOp(
        name=pvc_name,
        resource_name="newpvc",
        size="1Gi",
        modes=dsl.VOLUME_MODE_RWO
    )
    vgg16_op = func_to_container_op(
        func=tf_func,
        base_image="tensorflow/tensorflow:2.0.0-py3",
    )
    tensorboard_op = func_to_container_op(
        func=tensorboard_func,
        base_image="python:3.7-slim",
    )
    vgg16_task = vgg16_op(log_folder).add_pvolumes({
        log_folder:vop.volume,
    })
    tenaorboard_task = tensorboard_op(vgg16_task.outputs['logdir']).add_pvolumes({
        log_folder:vop.volume,
    })

In [11]:
kfp.compiler.Compiler().compile(vgg16_pipeline, 'transferlearing.zip')